# Übung 7.1: Vierecke zählen

Die Übungsaufgabe in dieser Woche ist von einer Aufgabe der [langen Nacht der Mathematik](https://mathenacht.de) inspiriert. Bei der Originalaufgabe ging es darum, die Anzahl der [Kongruenzklassen](https://de.wikipedia.org/wiki/Kongruenzabbildung) von Vierecken mit Flächeninhalt 1 auf einem Geobrett mit 4 x 4 Punkten 
![Geobrett](http://www.rittel-verlag.de/WebRoot/Store20/Shops/62618028/4C12/45CA/D328/2D5D/3895/C0A8/29B9/6EFD/Geobrett-4x4-Holz-01_m.jpg) zu zählen.

Die Einheiten sind dabei so normiert, dass die Fläche des kleinsten Quadrates auf dem Geobrett den Inhalt 1 hat.

## Flächeninhalt von Polygonen mit ganzzahligen Koordinaten

Für alle, die nicht in der Vorlesung waren: Der Flächeninhalt von Polygonen auf einem solchen Geobrett lässt sich sehr einfach über den [Satz von Pick](https://de.wikipedia.org/wiki/Satz_von_Pick) berechnen, indem man die Zahl $i$ der Gitterpunkte im Inneren (rote Punkte) und die Zahl $b$ der Punkte auf dem Rand des Polygons (grüne Punkte) zählt:
![Bild: Satz von Pick](https://upload.wikimedia.org/wikipedia/commons/f/f1/Pick-theorem.png)

Für den Flächeninhalt gilt dann
$$
A = i + \frac{b}{2} -1.
$$

Da die Vierecke auch konkav sein können (eine der Diagonalen liegt außerhalb des Vierecks), zerlegt man sie am besten entlang einer (inneren) Diagonale in zwei Dreiecke und verwendet den Code aus der Vorlesung für Dreiecke.

## Polygone sind kreuzungsfrei!

Bei Vierecken taucht ein Problem auf, dass wir in der Vorlesung in Bezug auf Dreiecke nicht hatten: Je nach Wahl der Punkte kann es vorkommen, dass sich gegenüberliegende Seiten schneiden. In diesem Fall bilden die Punkte kein gültiges Viereck.


## Kongruenz von Vierecken

Bei Vierecken reicht die Gleichheit der Seiten als Kriterium für die Kongruenz nicht aus. Ein hinreichendes Kriterium für die Kongruenz ist folgendes:

Zwei Vierecke $A_1A_2A_3A_4$ und $B_1B_2B_3B_4$ sind kongruent, wenn es eine zyklische Permutation $\sigma$ gibt, so dass gilt (ggf. nach Vertauschung von $B_2$ und $B_4$)
$$
\overline{A_iA_{i+1}} = \overline{B_{\sigma(i)}B_{\sigma(i+1)}} \quad \forall i \in \{ 1, 2, 3, 4 \}
$$
und
$$
\overline{A_iA_{i+2}} = \overline{B_{\sigma(i)}B_{\sigma(i+2)}} \quad \forall i \in \{ 1, 2 \},
$$
d.h. die Seiten und die Diagonalen sind gleich.

Die Prüfung geht am einfachsten, wenn man die Seiten in eine "kanonische" Reihenfolge bringt und dazu von den zyklischen Permutationen der Seiten `(a, b, c, d)` bzw. `(a, d, c, b)` die lexikografisch kleinste auswählt.

## Aufgabe
1. Bestimmen Sie – so wie wir es in der Vorlesung für Dreiecke getan haben – die Anzahl der Kongruenzklassen von Vierecken je Flächeninhalt.
2. Bestimmen Sie insbesondere die Anzahl der verschiedenen Vierecke mit Flächeninhalt 1 (Tipp: Es sind mehr als 12 und weniger als 20).

## Klassen aus der Vorlesung

Die in der Vorlesung gemeinsam programmierten Klassen für Vektoren und Dreiecke können Sie übernehmen:

In [ ]:
class Vector(tuple):
    """Beschreibt einen Vektor in der zweidimensionalen
    euklidischen Ebene"""
    
    def __new__(cls, *coords): 
        if len(coords) != 2: 
            raise ValueError(f"Dimension must be 2: {coords}")
        return super().__new__(cls, coords)
        
    def __add__(self, q):
        return Vector(self[0] + q[0], self[1] + q[1])
    
    def __sub__(self, q):
        return Vector(self[0] - q[0], self[1] - q[1])
    
    def __mul__(self, q):
        """Berechne das Skalarprodukt"""
        return self[0] * q[0] + self[1] * q[1]
    
    def normal(self):
        """Berechne den Normalenvektor"""
        return Vector(self[1], -self[0])    
    
    def len2(self):
        """Berechne das Quadrat der Länge des Vektors"""
        return self[0]*self[0] + self[1]*self[1]

In [1]:
class Triangle(tuple):
    """Beschreibt Dreiecke in der Ebene."""
    
    def _ensure_vector_(p):
        if isinstance(p, Vector): 
            return p 
        elif isinstance(p[0], tuple):
            return [ Triangle._ensure_vector_(q) for q in p[0]]
        else: 
            return Vector(*p)
    
    def __new__(cls, *points):
        """Erzeuge ein Dreieck aus den übergebenen Punkten.
        
        Die Punkte werden ggf. "positiv orientiert", damit wir 
        einfacher bestimmen können, ob ein Punkt im Inneren des
        Dreiecks liegt.
        """
        points = list(map(Triangle._ensure_vector_, points))
        
        # Check we have three points
        if len(points) != 3 or len(set(points)) != 3: 
            raise ValueError("Three different points needed to create a triangle")
        
        # Check that the points are not colinear and in positive order
        dot = (points[1] - points[0]).normal() * (points[2] - points[1])
        if dot == 0:
            raise ValueError(f"These points are on a line: {points}")
        elif dot < 0:
            # The points are in wrong order and need to be reversed
            points = reversed(points)
        
        return super().__new__(cls, points)
    
    def __init__(self, *points):
        """Initialisiere das Dreieck.
        
        Berechne die Seitenlängen.
        """        
        sides = defaultdict(int)
        normals = []
        for i in range(3):
            j = (i + 1) % 3
            sides[(self[j] - self[i]).len2()] += 1
            normals.append(((self[j] - self[i]).normal(), self[i]))
        
        self.sides = tuple(sorted(sides.items()))
        self.normals = tuple(normals)
        self.N = max(max([x for (x, y) in self]), max([y for (x, y) in self])) + 1
        
    def __repr__(self):
        return f"Triangle(sides: {self.sides}, points: {super().__repr__()}, max: {self.N})"
    
    def __hash__(self):
        return hash(self.sides)
    
    def __eq__(self, other):
        """Check if the other triangle is equal (congruent) 
        to this one.
        
        According to elementary geometry, this is the case iff
        the length of the three sides are identical.
        """
        return self.sides == other.sides
    
    def is_boundary(self, p):
        """Check if p is on the boundary of the triangle.
        
        This is the case, if there is a side rs with rp and rs being colinear
        and (p - r) * (p - s) <= 0."""
        for i in range(3):
            #print(f"{p} {self[i]} {i}")
            if p == self[i]:
                return True
            
            j = (i + 1) % 3
            if (p - self[i]) * (self[j] - self[i]).normal() != 0:
                continue
            else:
                return (p - self[i]) * (p - self[j]) <= 0
            
        return False
    
    def is_inner(self, p):
        """Check if p is an inner or boundary point.
        
        These points have a positive dot product with all normals.
        """
        for (n, q) in self.normals:
            #print(f"{p}: {n*(p-q)}")
            if n * (p-q) < 0:
                return False
    
        return True
 
    def area(self):
        """Calculate the area using Pick's theorem."""
        inner = []
        boundary = []
        for p in (Vector(x, y) for x in range(self.N) for y in range(self.N)):
            if self.is_boundary(p):
                boundary.append(p)
                
            elif self.is_inner(p):
                inner.append(p)
    
        return len(inner) + len(boundary)/2 - 1
